In [ ]:
#Initial conditions for the parallaxconst=(moon distance in Earth radii,geocentric RA,geocentric Dec)
parallaxinitialconst=[56.0,8.17,20.44]

#Test apparentSkyPosition
key="RD"
site=sites[key]
#location="Moon Center"
location="L1-21J"
apparentSkyPosition(parallaxinitialconst,site["lat"],site["lon"],site["alt"])

#Test fit parallax
parallaxsq(parallaxinitialconst,sites,"Moon Center",verbose=True)

In [ ]:
#Instrument properties
#LaLoma (San Vicente Ferrer, Colombia)
LALOMA_F=0.73
LALOMA_FL=2700.0 #mm
LALOMA_D=635 #mm
ZWO_COLS=4656.0 #px
ZWO_ROWS=3520.0 #px
ZWO_WIDTH=17.5 #mm
ZWO_HEIGHT=13.4 #mm

In [ ]:
#Theoretical spatial resolution of LaLoma telescope
pxsize=ZWO_WIDTH/(LALOMA_F*LALOMA_FL)/ZWO_COLS
pxsize_km=pxsize*MOON_DISTANCE
#At 70 degrees inclination
pxsize_inc=pxsize_km/np.cos(IMPACT_LOC["lat"]*DEG)
print(f"Theoretical angular resolution = {pxsize*180/np.pi*3600} arcsec")
print(f"Theoretical linear resolution = {pxsize_km} km")
print(f"Theoretical linear resolution (@{np.abs(IMPACT_LOC['lat'])} deg) = {pxsize_inc} km")

In [ ]:
solutionselenographic.x=np.array([-1.95449213e-02,  4.72321047e-03,  8.12963942e+00,  6.81466867e-02,
        2.73041416e-01,  2.02204853e+01, -2.50000867e+00, -7.99953324e-01])
print(selensq(solutionselenographic.x,site,LunarSites,verbose=True))

In [ ]:
def getradec(plateconst,x,y,xcen,ycen):
    """
    Get (RA,Dec) relative to center of frame from (X,Y) given plate constant parameters
    """
    dec = plateconst[3]*(x-xcen)+plateconst[4]*(y-ycen)+plateconst[5]
    ra = 15*math.cos(math.radians(dec))*(plateconst[0]*(x-xcen)-plateconst[1]*(y-ycen))+plateconst[2]
    return ra,dec
                         

def distsq(plateconst,stars=None,verbose=False):
    """
    Calculate distance squared for positions given plate constant and star coordinates
    """
    distsq=0
    drasq=[]
    ddecsq=[]
    center=stars.loc[stars["ObjName"]==REF_LUNARSITE].iloc[0]
    #print(center)
    for ind,star in stars.loc[stars["type"]=="Star"].iterrows():
        #print(star)
        ra,dec=getradec(plateconst,star["X"],star["Y"],center["X"],center["Y"])
        drasq+=[(ra-star["RA"])**2]
        ddecsq+=[(dec-star["DEC"])**2]
        distsq+=225*math.cos(math.radians(dec))*drasq[-1]+ddecsq[-1]
    if verbose:return distsq,drasq,ddecsq
    return distsq

def apparentSkyPosition(parallaxconst,lat,lon,ele,gst=GST):
    """
    Get apparent coordinate (RA,Dec) of observer at (lat,lon,ele) given the 
    parallaxconst=(moon distance in Earth radii,geocentric RA,geocentric Dec)
    """
    LST=gst+lon/15
    HA=(LST-parallaxconst[1])*15
    rhosin=0.996647*math.sin(math.atan(0.996647*math.tan(math.radians(lat))))+ele/6378160*math.sin(math.radians(lat))
    rhocos=math.cos(math.atan(0.996647*math.tan(math.radians(lat))))+ele/6378160*math.cos(math.radians(lat))
    deltara=math.degrees(math.atan(rhocos*math.sin(math.radians(HA))/(parallaxconst[0]*math.cos(math.radians(parallaxconst[2]))-rhocos*math.cos(math.radians(HA)))))
    ra=parallaxconst[1]-deltara/15
    dec=math.degrees(math.atan(math.cos(math.radians(HA+deltara))*((parallaxconst[0]*math.sin(math.radians(parallaxconst[2]))-rhosin)/(parallaxconst[0]*math.cos(math.radians(parallaxconst[2]))*math.cos(math.radians(HA))-rhocos))))
    return ra,dec


def parallaxsq(parallaxconst,sites=None,location=None,verbose=False):
    """
    Calculate Error Squared for each parallax correction
    """
    parallaxsq=0
    prasq=[]
    pdecsq=[]
    for key,site in sites.items():
        objs=site["data"]
        #print(key,objs.loc[objs["ObjName"]==location])
        loc=objs.loc[objs["ObjName"]==location].iloc[0]
        ra,dec=apparentSkyPosition(parallaxconst,site["lat"],site["lon"],site["alt"])
        prasq+=[(ra-loc["RA"])**2]
        pdecsq+=[(dec-loc["DEC"])**2]
        parallaxsq+=225*math.cos(math.radians(dec))*prasq[-1]+pdecsq[-1]
    if verbose: return parallaxsq,prasq,pdecsq
    return parallaxsq

# Get (RA,Dec) given selenographic coordinate (lat,lon)
def selenographictoradec(params,lat,lon):
    print(params,lat,lon)
    ra = params[0]*math.sin(math.radians(lon-params[6]))*math.cos(math.radians(lat-params[7]))+params[1]*math.sin(math.radians(lat-params[7]))+params[2]
    dec = params[3]*math.sin(math.radians(lon-params[6]))*math.cos(math.radians(lat-params[7]))+params[4]*math.sin(math.radians(lat-params[7]))+params[5]
    return ra,dec

# Calculates the error between selenographictoradec conversion and (raref,decref) given params
def selenerr(coord,raref,decref,params):
    ra,dec=selenographictoradec(params,coord[0],coord[1])
    return 225*math.cos(math.radians(dec))*(ra-raref)**2+(dec-decref)**2
    
# Calculate Error Squared for each RA,Dec conversion from selenographic coordinate
def selensq(params,site=None,lunarsites=None,verbose=False):
    selensq=0
    srasq=[]
    sdecsq=[]
    
    objs=site["data"]
    locations=objs.loc[objs["type"]=="Moon"]
    
    for ind in locations.index[2:]:
        location=locations.loc[ind]
        locname=location["ObjName"]
        sra=location["RA"]
        sdec=location["DEC"]
        lunarsite=lunarsites.loc[lunarsites["LunarSite"]==locname].iloc[0]
        if lunarsite["slon"]==-1:continue
        ra,dec=selenographictoradec(params,lunarsite["slat"],lunarsite["slon"])
        srasq+=[(ra-sra)**2]
        sdecsq+=[(dec-sdec)**2]
        selensq+=225*math.cos(math.radians(dec))*srasq[-1]+sdecsq[-1]
    if verbose: return selensq,srasq,sdecsq
    return selensq